In [1]:
import azure.cognitiveservices.speech as speechsdk

import pandas as pd

# Importing Sckit learn libraries for calculating Cosine Similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import json
import ast

ast.literal_eval

import os
from dotenv import load_dotenv

import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
credits_df = pd.read_csv("credits.csv")
movies_df = pd.read_csv("movies.csv")

# Shape of the dataset
print("Credits Shape:-", credits_df.shape)
print("Movies Data Shape:-", movies_df.shape)

Credits Shape:- (4803, 4)
Movies Data Shape:- (4803, 20)


In [3]:
credits_df.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
movies_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [5]:
# Merging both movies and credit df
movies_credits_df = movies_df.merge(credits_df, on="title")
movies_credits_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Converting into string format for JSON values


In [6]:
# changing the genres column from json to string
movies_credits_df["genres"] = movies_credits_df["genres"].apply(json.loads)
for index, i in zip(movies_credits_df.index, movies_credits_df["genres"]):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]["name"]))
    movies_credits_df.loc[index, "genres"] = str(list1)

In [7]:
# changing the keywords column from json to string
movies_credits_df["keywords"] = movies_credits_df["keywords"].apply(json.loads)
for index, i in zip(movies_credits_df.index, movies_credits_df["keywords"]):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]["name"]))
    movies_credits_df.loc[index, "keywords"] = str(list1)

In [8]:
# changing the production_companies column from json to string
movies_credits_df["production_companies"] = movies_credits_df[
    "production_companies"
].apply(json.loads)
for index, i in zip(movies_credits_df.index, movies_credits_df["production_companies"]):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]["name"]))
    movies_credits_df.loc[index, "production_companies"] = str(list1)

In [9]:
# changing the cast column from json to string
movies_credits_df["cast"] = movies_credits_df["cast"].apply(json.loads)
for index, i in zip(movies_credits_df.index, movies_credits_df["cast"]):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]["name"]))
    movies_credits_df.loc[index, "cast"] = str(list1)

In [10]:
# changing the production_countries column from json to string
movies_credits_df["production_countries"] = movies_credits_df[
    "production_countries"
].apply(json.loads)
for index, i in zip(movies_credits_df.index, movies_credits_df["production_countries"]):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]["name"]))
    movies_credits_df.loc[index, "production_countries"] = str(list1)

In [11]:
# changing the spoken_languages column from json to string
movies_credits_df["spoken_languages"] = movies_credits_df["spoken_languages"].apply(
    json.loads
)
for index, i in zip(movies_credits_df.index, movies_credits_df["spoken_languages"]):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]["name"]))
    movies_credits_df.loc[index, "spoken_languages"] = str(list1)

In [12]:
# changing the crew column from json to string

movies_credits_df["crew"] = movies_credits_df["crew"].apply(json.loads)


def director(x):
    for i in x:
        if i["job"] == "Director":
            return i["name"]


movies_credits_df["crew"] = movies_credits_df["crew"].apply(director)
movies_credits_df.rename(columns={"crew": "director"}, inplace=True)

In [13]:
movies_credits_df_backup = movies_credits_df.copy(deep=True)

movies_credits_df = movies_credits_df[
    ["id", "original_title", "genres", "cast", "director", "vote_average", "keywords"]
]
movies_credits_df.head()

,id,original_title,genres,cast,director,vote_average,keywords
0,19995,Avatar,"['Action', 'Adventure', 'Fantasy', 'Science Fi...","['Sam Worthington', 'Zoe Saldana', 'Sigourney ...",James Cameron,7.2,"['culture clash', 'future', 'space war', 'spac..."
1,285,Pirates of the Caribbean: At World's End,"['Adventure', 'Fantasy', 'Action']","['Johnny Depp', 'Orlando Bloom', 'Keira Knight...",Gore Verbinski,6.9,"['ocean', 'drug abuse', 'exotic island', 'east..."
2,206647,Spectre,"['Action', 'Adventure', 'Crime']","['Daniel Craig', 'Christoph Waltz', 'Léa Seydo...",Sam Mendes,6.3,"['spy', 'based on novel', 'secret agent', 'seq..."
3,49026,The Dark Knight Rises,"['Action', 'Crime', 'Drama', 'Thriller']","['Christian Bale', 'Michael Caine', 'Gary Oldm...",Christopher Nolan,7.6,"['dc comics', 'crime fighter', 'terrorist', 's..."
4,49529,John Carter,"['Action', 'Adventure', 'Science Fiction']","['Taylor Kitsch', 'Lynn Collins', 'Samantha Mo...",Andrew Stanton,6.1,"['based on novel', 'mars', 'medallion', 'space..."


In [14]:
movies_credits_df.isnull().sum()

id                 0
original_title     0
genres             0
cast               0
director          30
vote_average       0
keywords           0
dtype: int64

In [15]:
movies_credits_df.dropna(inplace=True)

## Removing unwanted characters from different list columns


In [16]:
def space(col):
    if col is None:
        return " "
    return str(col)


movies_credits_df["director"] = movies_credits_df["director"].apply(space)


movies_credits_df["genres"] = (
    movies_credits_df["genres"]
    .str.strip("[]")
    .str.replace(" ", "")
    .str.replace("'", "")
)

movies_credits_df["genres"] = movies_credits_df["genres"].str.split(",")


movies_credits_df["cast"] = (
    movies_credits_df["cast"]
    .str.strip("[]")
    .str.replace(" ", "")
    .str.replace("'", "")
    .str.replace('"', "")
)

movies_credits_df["cast"] = movies_credits_df["cast"].str.split(",")


def features(x):
    return (
        " ".join(x["keywords"])
        + " "
        + " ".join(x["cast"])
        + " "
        + x["director"]
        + " "
        + " ".join(x["genres"])
    )


movies_credits_df["combine_feature"] = movies_credits_df.apply(features, axis=1)

In [17]:
movies_credits_df.head()

,id,original_title,genres,cast,director,vote_average,keywords,combine_feature
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",James Cameron,7.2,"['culture clash', 'future', 'space war', 'spac...","[ ' c u l t u r e c l a s h ' , ' f u t u ..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",Gore Verbinski,6.9,"['ocean', 'drug abuse', 'exotic island', 'east...","[ ' o c e a n ' , ' d r u g a b u s e ' , ..."
2,206647,Spectre,"[Action, Adventure, Crime]","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",Sam Mendes,6.3,"['spy', 'based on novel', 'secret agent', 'seq...","[ ' s p y ' , ' b a s e d o n n o v e l ..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[ChristianBale, MichaelCaine, GaryOldman, Anne...",Christopher Nolan,7.6,"['dc comics', 'crime fighter', 'terrorist', 's...","[ ' d c c o m i c s ' , ' c r i m e f i ..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction]","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",Andrew Stanton,6.1,"['based on novel', 'mars', 'medallion', 'space...","[ ' b a s e d o n n o v e l ' , ' m a r ..."


In [18]:
cv = CountVectorizer(stop_words="english")  # creating new CountVectorizer() object
count_matrix = cv.fit_transform(
    movies_credits_df["combine_feature"]
)  # feeding combined strings(movie contents) to CountVectorizer() object
cosine_sin = cosine_similarity(count_matrix, count_matrix)

In [19]:
# Reset index of our main DataFrame and construct reverse mapping as before
movies_credits_df = movies_credits_df.reset_index()
indices = pd.Series(movies_credits_df.index, index=movies_credits_df["original_title"])

In [20]:
# Function that takes in movie title as input and outputs most similar movies
def get_movie_recommendation(title, cosine_sim=cosine_sin):
    try:
        # Get the index of the movie that matches the title
        idx = indices[title]

        # Get the pairwsie similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Sort the movies based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the scores of the 10 most similar movies
        sim_scores = sim_scores[1:11]

        # Get the movie indices
        movie_indices = [i[0] for i in sim_scores]

        # Return the top 10 most similar movies

        print("-----------Movies-Recommended-----------------")
        print("---" * 15)
        return movies_credits_df["original_title"].iloc[movie_indices]
    except Exception:
        return "I'm not sure how to respond to that."

In [21]:
# get service secrets
load_dotenv()
ss_endpoint = os.environ.get("ss_endpoint")
ss_key = os.environ.get("ss_key")
ls_endpoint = os.environ.get("ls_endpoint")
ls_key = os.environ.get("ls_key")
region = os.environ.get("region")

In [22]:
speech_config = speechsdk.SpeechConfig(subscription=ss_key, region=region)
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)

In [31]:
def speech_input(b):
    print("Listening....")
    result = speech_recognizer.recognize_once()

    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        recognized_text = result.text.rstrip(
            "." "."
        )  # Remove dot from the end of recognized text

        print("Recognized: {}".format(recognized_text))
        print(get_movie_recommendation(recognized_text))
    elif result.reason == speechsdk.ResultReason.NoMatch:
        raise Exception(
            "No speech could be recognized: {}".format(result.no_match_details)
        )
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        raise Exception(
            "Speech Recognition canceled: {}".format(cancellation_details.reason)
        )


def on_enter_key_pressed(change):
    if change["name"] == "value" and change["new"]:
        user_input = change["new"].replace("\n", "")  # Remove the newline character
        print("Recognized: {}".format(user_input))
        print(get_movie_recommendation(user_input))
        # text_input.value = ""  # Clear the text input after processing the input

In [32]:
def bot():

    def refresh_output(b):
        clear_output(wait=True)
        bot()

    def get_text(b):
        display(text_input)

    refresh_button = widgets.Button(description="Refresh Output")
    button1 = widgets.Button(description="Text")
    button2 = widgets.Button(description="Audio")
    text_input = widgets.Text(placeholder="Enter your input", continuous_update=False)

    # Assign the function to be called when the button is clicked
    refresh_button.on_click(refresh_output)
    button1.on_click(get_text)
    button2.on_click(speech_input)
    text_input.observe(
        on_enter_key_pressed
    )  # Assign the function to be called when Enter is pressed in the text input box

    # Attach the function to the button's click event
    display(refresh_button)
    display(button1)
    display(button2)


bot()

Button(description='Refresh Output', style=ButtonStyle())

Button(description='Text', style=ButtonStyle())

Button(description='Audio', style=ButtonStyle())

Listening....
Recognized: The Wolverine
-----------Movies-Recommended-----------------
---------------------------------------------
511                          X-Men
203                             X2
33           X-Men: The Last Stand
1231                    The Shadow
1017                Kate & Leopold
46      X-Men: Days of Future Past
1654          Dragonball Evolution
1934                        Sheena
581        Star Trek: Insurrection
1297                  Superman III
Name: original_title, dtype: object
